In [282]:
%%capture
!pip install transformers==4.2.1
!pip install tensorflow-gpu==2.3.0 
print('Done!')

In [283]:
!pip install spacy==3.0

In [284]:
!python -m spacy download es_core_news_md

2021-05-19 10:41:44.729082: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [285]:
%reset
import pandas as pd
import tensorflow as tf

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [286]:
train_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/haha_2021_train.csv", sep=',')
train_dataframe[train_dataframe.is_humor==1]

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,1,2,2,0,0,0,1.5000,NaN,NaN
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,1,3,0,1,0,0,1.5000,NaN,NaN
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,0,2,1,0,1,1,2.6000,NaN,NaN
6,tweet7,"—Buenas don Pepe, ¿me vende un litro de leche?...",1,1,1,1,1,1,0,2.5000,NaN,NaN
12,tweet13,#20CosasQueHacerAntesDeMorir: Enseñarles la di...,1,0,1,1,2,1,0,2.6000,reference,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
23991,tweet23992,¿Cuál es el dominio mas erótico?... www.pet.es...,1,2,2,1,0,0,0,1.3333,NaN,NaN
23993,tweet23994,"—¿Nivel de inglés? —100% —Traduzca ""Escoger un...",1,0,2,4,0,0,0,1.6667,wordplay,NaN
23994,tweet23995,"Era tan, pero tan mal jugador, que hizo un gol...",1,1,0,2,0,2,0,3.0000,NaN,NaN
23995,tweet23996,"—Estoy afuera de tu casa, sal. —Ya voy, pimienta.",1,0,2,1,1,1,0,2.2000,NaN,NaN


In [287]:
mechanisms = train_dataframe[train_dataframe.is_humor==1].dropna()

In [288]:
mechanisms

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
21,tweet22,"-El alcohol es un veneno. –Si es un veneno, ¿P...",1,2,1,1,0,1,0,2.333300,reference,substance use
39,tweet40,"–¿Tiene pastillas para la diarrea? –No. –Ok, d...",1,0,1,1,2,1,0,2.600000,embarrassment,health
63,tweet64,4 etapas de la vida: 1- Creer en Santa Claus. ...,1,0,0,3,1,1,0,2.600000,analogy,age
70,tweet71,-¡Policía! Dos chicas se están peleando por mí...,1,0,7,2,11,2,2,2.583333,misunderstanding,body shaming; women
73,tweet74,Teníamos una farmacia pero la cerramos porque ...,1,0,1,1,1,1,2,3.333300,wordplay,professions
...,...,...,...,...,...,...,...,...,...,...,...,...
23899,tweet23900,—Oye Peña Nieto ¿Quién crees que sea el suceso...,1,0,4,1,0,0,0,1.200000,misunderstanding,professions
23932,tweet23933,QUE LE DIJO EL BORRACHO AL RECIEN NACIDO : BEB...,1,0,4,0,1,0,0,1.400000,wordplay,substance use
23949,tweet23950,¿Ponerme triste porque te caes? Jajaja! #NiQue...,1,1,3,2,0,0,0,1.400000,analogy,technology
23967,tweet23968,La ex novia de Jaimito se quería lanzar desde ...,1,1,3,0,2,0,0,1.800000,wordplay,women


In [289]:
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import numpy as np

In [290]:
tar = LabelEncoder().fit_transform(mechanisms["humor_target"])

In [291]:
tar

array([49, 25,  0, ..., 50, 52, 25])

In [292]:
# Hay que ponderar categorías, aunque me da más miedo el tema de la tokenizacion
categories = sorted(np.unique(tar), reverse=False)
hist= Counter(tar) 
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

In [293]:
import spacy
import es_core_news_md

In [294]:
mechanisms["text"][23932]

'QUE LE DIJO EL BORRACHO AL RECIEN NACIDO : BEBE \n#chistes #humor'

In [295]:
test = mechanisms["text"][23932]

In [296]:
nlp = es_core_news_md.load()

In [297]:
doc_analysis = nlp(test)

In [298]:
dataframe = pd.DataFrame({"Palabra": [], "Lema": [], "Categoria_gramatical": []})

for token in doc_analysis:
    dataframe = dataframe.append({"Palabra": token.text, "Lema": token.lemma_, "Categoria_gramatical": token.pos_}, ignore_index=True)
  
dataframe

,Palabra,Lema,Categoria_gramatical
0,QUE,que,SCONJ
1,LE,él,PRON
2,DIJO,DIJO,PROPN
3,EL,el,DET
4,BORRACHO,BORRACHO,PROPN
5,AL,AL,ADP
6,RECIEN,recien,NOUN
7,NACIDO,nacido,ADJ
8,:,:,PUNCT
9,BEBE,bebe,NOUN


Noun-based approach

In [299]:
XSamples = []
for sample in mechanisms["text"]:
  doc_analysis = nlp(sample)
  sample = ""
  for token in doc_analysis:
    if token.pos_ == "NOUN" or token.pos_ == "ADJ":
      sample += token.lemma_ + " "
  
  if sample == '':
    sample += "nada"
  XSamples.append(sample)

XSamples

['alcohol veneno veneno cosa ',
 'pastilla diarrea rollo papel higiénico ',
 'etapa vida 2- ',
 'chica -¿y problema gordo ',
 'farmacia remedio # ',
 'mujer jaimito cosa cielo -ay q lindo cara vuelta ',
 'nada',
 'vez momento difícil situación día ',
 'día suegra madre político política mierda ',
 'presidente hario casa gobierno celeste chiste humor ',
 'mujer mano maestro duda levantáram maldito cerdo panocha ',
 'bateria celular ',
 'vida # humortico ',
 'grande cerda mamá dile ',
 'cupido ',
 'día café empanada café empanada ',
 'puta puta sordo ',
 'tweet día ',
 'sombra peso mierda ',
 'condenado muchacha demonio c.a.n.s.a.d.a. ',
 'virus mamá ',
 'típico taxi táctica escape taxista ',
 'perro argentino chiste humor ',
 'juego mesa familia secretaría ',
 'cosa amor ',
 'situación sentimental único persona corazón traficange órgano ',
 'mujer proteja payaso ninja ',
 'secuestrado suegra qu ',
 'mundo caro fb ',
 'caloría lata secreto lata ',
 'clima gana día acostado clima ',
 'mam

In [300]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from nltk.stem.snowball import SpanishStemmer

In [301]:
def spanish_stemmer(sentence):
    stemmer = SpanishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word',
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))

In [302]:
vectorizer = CountVectorizer(analyzer=spanish_stemmer)
X = vectorizer.fit_transform(XSamples)

In [303]:
X.shape

(1629, 2225)

In [304]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [305]:
target = LabelEncoder().fit_transform(mechanisms["humor_target"])
Y = to_categorical(target)

In [306]:
np.unique(target)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52])

In [307]:
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Definir una red sencilla, mucho ojo porque hay pocos datos y las categorías están superdesbalanceadas

In [308]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [309]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1Score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [319]:
input = Input(2225,)
x = Dense(128, activation="relu")(input)
x = Dropout(0.25)(x)
x = Dense(64, activation="sigmoid")(x)
x = Dropout(0.25)(x) 
x = BatchNormalization()(x)
output = Dense(53, activation="softmax")(x)

model = Model([input], output)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=['accuracy', f1Score])
model.summary()

Model: "functional_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 2225)]            0         
_________________________________________________________________
dense_98 (Dense)             (None, 128)               284928    
_________________________________________________________________
dropout_51 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_99 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_52 (Dropout)         (None, 64)                0         
_________________________________________________________________
batch_normalization_48 (Batc (None, 64)                256       
_________________________________________________________________
dense_100 (Dense)            (None, 53)              

In [320]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [321]:
X_train, X_test, Y_train, Y_test = train_test_split(X.toarray(), Y, test_size=0.2)

In [322]:
y_train_classes = np.argmax(Y, axis=1)
 
pesos = class_weight.compute_class_weight('balanced',
                                          np.unique(y_train_classes),
                                          y_train_classes)
pesos = dict(enumerate(pesos))  

In [314]:
X_train.shape

(1303, 2225)

In [315]:
Y_train.shape

(1303, 53)

In [316]:
X_test.shape

(326, 2225)

In [317]:
Y_test.shape

(326, 53)

In [323]:
model.fit(X_train, Y_train, batch_size=16, validation_data=(X_test, Y_test), class_weight=pesos, epochs=50)

Epoch 1/50
82/82 [==============================] - 0s 6ms/step - loss: 4.4226 - accuracy: 0.0130 - f1Score: 0.0000e+00 - val_loss: 4.0165 - val_accuracy: 0.0153 - val_f1Score: 0.0000e+00
Epoch 2/50
82/82 [==============================] - 0s 4ms/step - loss: 4.3906 - accuracy: 0.0184 - f1Score: 0.0000e+00 - val_loss: 3.9887 - val_accuracy: 0.0061 - val_f1Score: 0.0000e+00
Epoch 3/50
82/82 [==============================] - 0s 4ms/step - loss: 4.2718 - accuracy: 0.0246 - f1Score: 0.0000e+00 - val_loss: 3.9518 - val_accuracy: 0.0092 - val_f1Score: 0.0000e+00
Epoch 4/50
82/82 [==============================] - 0s 4ms/step - loss: 4.0851 - accuracy: 0.0246 - f1Score: 0.0000e+00 - val_loss: 3.9128 - val_accuracy: 0.0276 - val_f1Score: 0.0000e+00
Epoch 5/50
82/82 [==============================] - 0s 4ms/step - loss: 3.7659 - accuracy: 0.0269 - f1Score: 0.0000e+00 - val_loss: 3.8863 - val_accuracy: 0.0429 - val_f1Score: 0.0000e+00
Epoch 6/50
82/82 [==============================] - 0s 4ms/s